# Input/Output data integration and harmonisation - CZ healthcare services

Prepared by [**J.Grazzini**](mailto:jacopo.grazzini@ec.europa.eu) ([_Eurostat_](https://ec.europa.eu/eurostat)).

## Introduction

This notebook illustrates some basic usage of the [`pyeudatanat`](https://github.com/eurostat/pyEUDatNat) and [`pyeufacility`](https://github.com/eurostat/pyeufacility) packages in order to ingest and harmonise data collected from national providers. 

The example presented herein aims at formatting CZ data on healthcare services according to an agreed [common template](https://github.com/eurostat/basic-services/blob/master/data/healthcare/metadata.pdf).

**Table of contents**

* [Setting and checking the environment](#environment).
* [Definition of the output configuration format](#configuration).
* [Load and explore the input data](#exploration).
* [Fit the input to the output configuration format](#matching).
* [Store and expose the output harmonised data](#store-data).
* [Set and share the formatting metadata](#store-metadata).
* [Adopt a metadata-based approach](#metadata-approach).
* [Automated metadata-based harmonisation](#harmonisation).
* [Automated metadata-based validation](#validation).

## Setting and checking the environment <a id="environment"></a>

Let's run some setup necessary to import the dependency(ies) required to run the project... 

In [1]:
thisdir = !pwd # os.getcwd()
PROJECT = 'basic-services'

import os, sys
import numpy as np
import pandas as pd
import json
from pprint import pprint as pprint

First, you will need to import the `pyeudatnat` package that contains various useful functions/methods: 

In [2]:
try:
    import pyeudatnat
except ImportError:
    try:
        !{sys.executable} -m pip install git+https://github.com/eurostat/pyEUDatNat.git
        # !conda install --yes --prefix {sys.prefix} git+https://github.com/eurostat/pyEUDatNat.git
    except:
        raise IOError("sorry, you're doomed, you won't be able to run this notebook...")
    else:
        import pyeudatnat
finally:
    print('package pyeudatnat available to run project %s' % PROJECT)
    from pyeudatnat import misc, io, meta, text

package pyeudatnat available to run project basic-services


Then, you will need to load the [`pyeufacility`](https://github.com/eurostat/basic-services/tree/master/src/python/pyeufacility) module available with the [`'basic-services'`](https://github.com/eurostat/basic-services/) project.
The following is required only if `pyeufacility` is not remotely installed/available from `pypi`, but it will be working only if you have a local version on your machine. Another simple option is to update your environment variable `PYTHONPATH`. 

In [3]:
try:
    import pyeufacility
except ImportError:
    try:
        assert 'pyeufacility' in [mod.__name__ for mod in sys.modules.values()]
    except AssertionError:
        # first case: this notebook will need to be run from the project directory, otherwise...
        try:
            pardir = os.path.abspath(os.path.join(thisdir[0], '../'))
            assert 'pyeufacility' in os.listdir(pardir)
        except:
            # second case: install from github
            try:
                !{sys.executable} -m pip install git+https://github.com/eurostat/basic-services.git#egg=pyeufacility\&subdirectory=src/python
            except:
                raise IOError("sorry, you're doomed, you won't be able to run this notebook...")
            else:
                import pyeufacility
        else:
            sys.path.insert(0,pardir)     
            import pyeufacility
finally:
    print('package pyeufacility available to run project %s' % PROJECT)
    from pyeufacility import config

package pyeufacility available to run project basic-services


We will also use the following variable for handling the paths to the local data repositories:

In [4]:
try:
    PACKPATH = getattr(sys.modules['pyeufacility'], 'PACKPATH')
except:
    PACKPATH = getattr(sys.modules['pyeufacility'], '__path__')[0]

Let's check some basic global parameters/metadata already made available there...

In [5]:
print("\033[4m\033[1mCountries\033[0m considered for harmonisation:\033[0m %s" % list(pyeudatnat.COUNTRIES))
print("\033[4m\033[1mFacilities\033[0m available for harmonisation:\033[0m %s" % pyeufacility.FACILITIES)

Countries considered for harmonisation: ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', 'CY', 'AT', 'SE', 'IE', 'LV', 'PL', 'UK', 'IS', 'NO', 'CH', 'LI']
Facilities available for harmonisation: {'HCS': {'code': 'hcs', 'name': 'Healthcare services'}, 'EDU': {'code': 'edu', 'name': 'Educational facilities'}, 'Oth': {'code': 'other', 'name': 'Other basic services TBD'}}


## Definition of the output configuration format <a id="configuration"></a> 

Next, we import the `config` module, and check also some of the metadata made available through the `pyeufacility` module. 

In particular, the format of the output harmonised dataset, such as the output columns (name and type), the output encoding format, the separator for output CSV, _etc_..., are defined for any given type of facility, _e.g._ HealthCare Services (*HCS*), separately in the global dictionary `config.FACMETADATA`:

In [6]:
print("Fields of the configuration template: \033[1m%s\033[0m" 
      % list(config.FACMETADATA['HCS'].keys()))
print("Formatting of output data is available for 'HCS': \033[1m%s\033[0m" 
      % config.FACMETADATA['HCS']['category'])

Fields of the configuration template: ['path', 'info', 'options', 'index', 'facil', 'category']
Formatting of output data is available for 'HCS': {'code': 'hcs', 'name': 'Healthcare services'}


In [7]:
config.FACMETADATA['HCS']

{'path': '../../../data/healthcare',
 'info': 'metadata.pdf',
 'options': {'lang': 'en',
  'fmt': {'geojson': 'geojson', 'csv': 'csv', 'gpkg': 'gpkg'},
  'proj': None,
  'sep': ',',
  'encoding': 'utf-8',
  'dtfmt': '%d/%m/%Y'},
 'index': {'id': {'name': 'id',
   'desc': 'The healthcare service identifier - This identifier is based on national identification codes, if it exists.',
   'type': 'int',
   'values': None},
  'name': {'name': 'hospital_name',
   'desc': 'The name of the healthcare institution',
   'type': 'str',
   'values': None},
  'site': {'name': 'site_name',
   'desc': 'The name of the specific site or branch of a healthcare institution',
   'type': 'str',
   'values': None},
  'lat': {'name': 'lat',
   'desc': 'Latitude (WGS 84)',
   'type': 'float',
   'values': None},
  'lon': {'name': 'lon',
   'desc': 'Longitude (WGS 84)',
   'type': 'float',
   'values': None},
  'geo_qual': {'name': 'geo_qual',
   'desc': 'A quality indicator for the geolocation - 1: Good, 2: Med

Using this global variable, it is possible to retrieve the specific formatting of *HCS* is described through the contents of `config.FACMETADATA['HCS']` that describes the columns/fields that will be available in the output dataset:

In [8]:
print("\033[4mConfiguration template:\033[0m")
# print(json.dumps(config.CONFIGINFO['HCS'], indent=2))
pprint(config.FACMETADATA['HCS'])

Configuration template:
{'category': {'code': 'hcs', 'name': 'Healthcare services'},
 'facil': 'HCS',
 'index': {'ER': {'desc': "Flag 'yes/no' for whether the healthcare site "
                          'provides emergency medical services',
                  'name': 'emergency',
                  'type': 'str',
                  'values': ['yes', 'no']},
           'PP': {'desc': "Status 'private/public' of the healthcare service",
                  'name': 'public_private',
                  'type': 'str',
                  'values': ['public', 'private']},
           'beds': {'desc': 'Measure of capacity by number of beds (most '
                            'common)',
                    'name': 'cap_beds',
                    'type': 'int',
                    'values': None},
           'cc': {'desc': 'Country code (ISO 3166-1 alpha-2 format)',
                  'name': 'cc',
                  'type': 'str',
                  'values': ['BE',
                             'EL',
   

This provides for instance with the definitions of the desired output fields, for instance:

In [9]:
print("- output attribute \033[1m'%s'\033[0m defined as:\n\033[1m%s\033[0m" % 
      (config.FACMETADATA['HCS']['index']['site']['name'], config.FACMETADATA['HCS']['index']['site']))
print("- output attribute \033[1m'%s'\033[0m defined as:\n\033[1m%s\033[0m" % 
      (config.FACMETADATA['HCS']['index']['lat']['name'], config.FACMETADATA['HCS']['index']['lat']))

- output attribute 'site_name' defined as:
{'name': 'site_name', 'desc': 'The name of the specific site or branch of a healthcare institution', 'type': 'str', 'values': None}
- output attribute 'lat' defined as:
{'name': 'lat', 'desc': 'Latitude (WGS 84)', 'type': 'float', 'values': None}


Note that the parameters of any given facility, _e.g._ *HCS*, as defined in `config.FACMETADATA` can actually be (re)set through an external configuration JSON file, _e.g._ *hcs.json*:

In [10]:
cfg = os.path.join(PACKPATH, 'hcs.json')
with open(cfg, 'r') as fp:
    metadata = json.load(fp)
print("Configuration template is parsed through \033[1m'%s'\033[0m file:" 
      % cfg.split(PROJECT)[1])
# metadata
print(json.dumps(metadata, indent=2))

Configuration template is parsed through '/src/python/pyeufacility/hcs.json' file:
{
  "path": "../../../data/healthcare",
  "info": "metadata.pdf",
  "options": {
    "lang": "en",
    "fmt": {
      "geojson": "geojson",
      "csv": "csv",
      "gpkg": "gpkg"
    },
    "proj": null,
    "sep": ",",
    "encoding": "utf-8",
    "dtfmt": "%d/%m/%Y"
  },
  "index": {
    "id": {
      "name": "id",
      "desc": "The healthcare service identifier - This identifier is based on national identification codes, if it exists.",
      "type": "int",
      "values": null
    },
    "name": {
      "name": "hospital_name",
      "desc": "The name of the healthcare institution",
      "type": "str",
      "values": null
    },
    "site": {
      "name": "site_name",
      "desc": "The name of the specific site or branch of a healthcare institution",
      "type": "str",
      "values": null
    },
    "lat": {
      "name": "lat",
      "desc": "Latitude (WGS 84)",
      "type": "float",
    

## Load and explore the input data<a id="exploration"></a> 

We require first to import some basic constructor utilities from the `pyeufacility.config` module: 

In [11]:
from pyeufacility.config import facilityFactory

Let's consider a simple example: as raw information in the original table, *CZ* data is already available with the lat/lon geographical coordinates encoded in a single column of the input data (named `'GPS`'). "Integrating" these data is nothing else than extracting the coordinates, reshufling some of the columns and dumping it in a new table.

To do so, it will be necessary to retrieve the structure (in terms of columns and data format) of the original table and "match" it to the output desired structure, as defined in the previously mentioned `config.CONFIGINFO` parameter. 

Let's first defined the input structure using the class constructor `facilityFactory`:

In [12]:
CZhcs = facilityFactory(fac='HCS', country = 'CZ')

print(CZhcs.COUNTRY)
print(CZhcs.CATEGORY)

{'CZ': 'Czechia'}
{'code': 'hcs', 'name': 'Healthcare services'}


and define the *CZ* data as an instance of the previously dynamically defined class `CZhcs`, with specific details regarding the data, such as the location of the original table:

In [13]:
cz = CZhcs(file = 'export-2020-02.csv', 
           path = os.path.abspath('../../../data/healthcare/raw/'), 
           lang = 'cs')

print("- source file: \033[1m'%s'\033[0m" % cz.file.split(PROJECT)[1])
print("- country code: \033[1m'%s'\033[0m" % cz.cc)
print("- language used in the original table: \033[1m'%s'\033[0m" % cz.lang)

- source file: '/data/healthcare/raw/export-2020-02.csv'
- country code: 'CZ'
- language used in the original table: 'cs'


Incidentally, the output template configuration `config.FACMETADATA['HCS']` (*e.g.*, list of desired columns in the output data, path of the output file, projection considered for the output data, *etc...*) is also loaded into the newly created `cz` instance and made available through its `config` attribute:

In [14]:
assert cz.config.to_dict() == config.FACMETADATA['HCS']

Overall, this is enough to load the data using the `load_data` method which stores the input data as a [`pandas`](https://pandas.pydata.org) dataframe structure in the `data` attribute of the `cz` instance. 

At this stage, it is necessary to check the input data, _e.g._ let's have a look at the column names and the content of the table in the raw dataset: obviously, some formatting is needed. Related information is parsed to the loading  `load_data` method through the implicit attributes of the `options['load']` of the `cz` instance:

In [15]:
# first incomplete try
try:
    cz.load_data()
except:
    print('Further information regarding data loading is requested...')
    cz.options['load'].update({'encoding': 'latin1', 'sep': ';'})
    # rerun...
    cz.load_data() 
    # note: we could have run: cz.load_data(encoding='latin1', sep=;') without the prior setting

cz.data.head(5)

Further information regarding data loading is requested...


,ZdravotnickeZarizeniId,PCZ,PCDP,NazevCely,DruhZarizeni,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,...,PscSidlo,ObecSidlo,UliceSidlo,CisloDomovniOrientacniSidlo,OborPece,FormaPece,DruhPece,OdbornyZastupce,GPS,LastModified
0,138130,0,1,Zubní studio V+V s.r.o.,Samostatná ordinace PL - stomatologa,Pelhøimov,39301.0,Komenského,1465,Kraj Vysoèina,...,39301.0,Pelhøimov,Pod Náspem,641,"zubní lékaøství, Dentální hygienistka","primární ambulantní péèe, specializovaná ambul...",NaN,"Veronika kodová, Vít koda",49.427787867131 15.218289906425,01-02-2020 23:03
1,138129,0,4,TECTUM spol. s r.o.,Odbìrová místnost,Most,43401.0,Topolová,1234,Ústecký kraj,...,36001.0,Karlovy Vary,Bezruèova,1098/10,klinická biochemie,specializovaná ambulantní péèe,NaN,"David Hepnar, JITKA PODROUKOVÁ",50.497217046878 13.650265371038,01-02-2020 23:03
2,138128,1,0,"MUDr. Milan Kuèera, s.r.o.",Samostatná ordinace lékaøe specialisty,Praha,15000.0,Kartouzská,3274/10,Hlavní mìsto Praha,...,27201.0,Kladno,T. G. Masaryka,2104,dermatovenerologie,ambulantní péèe,NaN,Jan Kuèera,50.073416715218 14.401090995315,01-02-2020 23:03
3,138127,1,0,Tereza Horáèková,Výdejna zdravotnických prostøedkù,Praha,18200.0,Klapkova,154/46,Hlavní mìsto Praha,...,NaN,NaN,NaN,NaN,praktické lékárenství,NaN,lékárenská péèe,Petra Holasová,50.126760922745 14.456956313382,01-02-2020 23:03
4,138126,0,0,Veobecný lékaø Prevent s.r.o.,Samost. ordinace veob. prakt. lékaøe,Telè,58856.0,Masarykova,330,Kraj Vysoèina,...,28163.0,Kozojedy,1. máje,67,"veobecné praktické lékaøství, veobecné prakt...","primární ambulantní péèe, zdrav. péèe poskytov...",NaN,ARNOT STANÌK,49.183499865155 15.45981410537,01-02-2020 23:03


More options are obviously available, as allowed by the `pandas` methods, for instance [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html) or [`read_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html), depending on the format of the input data you are dealing with. 

Now, basic information regarding the input data can be retrieved through the `data` attribute of the `cz` instance which is a common `pandas` dataframe. For instance, if we are interested in the list of columns:

In [16]:
print("\033[4mInput dataset columns:\033[0m")
print(cz.data.columns)

Input dataset columns:
Index(['ZdravotnickeZarizeniId', 'PCZ', 'PCDP', 'NazevCely', 'DruhZarizeni',
       'Obec', 'Psc', 'Ulice', 'CisloDomovniOrientacni', 'Kraj', 'KrajCode',
       'Okres', 'OkresCode', 'SpravniObvod', 'PoskytovatelTelefon',
       'PoskytovatelFax', 'DatumZahajeniCinnosti',
       'IdentifikatorDatoveSchranky', 'PoskytovatelEmail', 'PoskytovatelWeb',
       'PoskytovatelNazev', 'Ico', 'TypOsoby', 'PravniFormaKod',
       'KrajCodeSidlo', 'KrajSidlo', 'OkresCodeSidlo', 'OkresSidlo',
       'PscSidlo', 'ObecSidlo', 'UliceSidlo', 'CisloDomovniOrientacniSidlo',
       'OborPece', 'FormaPece', 'DruhPece', 'OdbornyZastupce', 'GPS',
       'LastModified'],
      dtype='object')


To make the identification of the appropriate columns in the input table, we can (approximately) translate the column names from *'cs'* to *'en'*, using the [`googletrans`](https://pypi.org/project/googletrans/) package. Note that you may run into [this issue](https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group) with the abovementioned package. Try to rerun the cell for a dirty fix, or follow the instructions provided in the link (*e.g.*, install the [`py-googletrans`](https://github.com/BoseCorp/py-googletrans) package):


In [17]:
print("\033[4mColumn names translated in English:\033[0m")
print(cz.get_cols(olang = 'en'))

Column names translated in English:
['Medical Equipment Id', 'PCZ', 'PCDP', 'CellName', 'Type of device', 'Village', 'Zip code', 'Street', 'Home Number Orientation', 'Region', 'KrajCode', 'District', 'OkresCode', 'Administrative District', 'Phone Provider', 'ProviderFax', 'Date Started Activities', 'Data Box Identifier', 'Email Provider', 'Web Provider', 'ProviderName', 'Ico', 'TypePersons', 'LegalFormCode', 'KrajCodeSidlo', 'KrajSidlo', 'OkresCodeSidlo', 'OkresSidlo', 'PscSidlo', 'ObecSidlo', 'StreetSidlo', 'HomeNumberOrientRescription', 'OborPece', 'FormaPece', 'TypePece', 'Professional Representatives', 'GPS', 'LastModified']


*update on 1/12/20*: The problem `error in result (AttributeError: 'NoneType' object has no attribute 'group')` reemerged recently as reported in [this ticket](https://github.com/ssut/py-googletrans/issues/234) with a possible [fix](https://github.com/ssut/py-googletrans/pull/237) (also discussed [here](https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group)). Issue has been solved with an updated version of the `googletrans` package:
```python
pip install googletrans==4.0.0-rc1
```
Well, this did not go very well... The translation is actually made difficult because of the formatting of the column names (merged words). To improve the translation output, we add a filter (split of words on capital letters) on the text to be translated, namely: 

In [18]:
print("\033[4mPrior filtered column names translated in English:\033[0m")
print(cz.get_cols(olang = 'en', filt = text.TextProcess.split_at_upper, force = True))

Prior filtered column names translated in English:
['Medical Equipment Id', 'PCZ', 'PCDP', 'The name of the cell', 'Type of device', 'Village', 'Zip code', 'Street', 'Home Number Orientation', 'Region', 'County Code', 'District', 'District Code', 'Administrative District', 'Phone Provider', 'Fax Provider', 'Date Started Activities', 'Mailbox Identifier', 'Email Provider', 'Web Provider', 'Provider Name', 'Ico', 'Type of Person', 'Legal Form Code', 'Country Code Seat', 'County Seat', 'District Code Headquarters', 'District Sidlo', 'Psc Seat', 'The village of Sidlo', 'Street Sidlo', 'Home Orientation Number', 'Field Furnaces', 'Furnace Form', 'Furnace Type', 'Professional Representative', 'GPS', 'Last Modified']


Note that the translated column names are stored in the `cols` attributes of the instance:

In [19]:
print("\033[4mColumn names in original 'cs' language and available 'en' translation:\033[0m")
pprint(cz.cols)

Column names in original 'cs' language and available 'en' translation:
[{'cs': 'ZdravotnickeZarizeniId', 'en': 'Medical Equipment Id'},
 {'cs': 'PCZ', 'en': 'PCZ'},
 {'cs': 'PCDP', 'en': 'PCDP'},
 {'cs': 'NazevCely', 'en': 'The name of the cell'},
 {'cs': 'DruhZarizeni', 'en': 'Type of device'},
 {'cs': 'Obec', 'en': 'Village'},
 {'cs': 'Psc', 'en': 'Zip code'},
 {'cs': 'Ulice', 'en': 'Street'},
 {'cs': 'CisloDomovniOrientacni', 'en': 'Home Number Orientation'},
 {'cs': 'Kraj', 'en': 'Region'},
 {'cs': 'KrajCode', 'en': 'County Code'},
 {'cs': 'Okres', 'en': 'District'},
 {'cs': 'OkresCode', 'en': 'District Code'},
 {'cs': 'SpravniObvod', 'en': 'Administrative District'},
 {'cs': 'PoskytovatelTelefon', 'en': 'Phone Provider'},
 {'cs': 'PoskytovatelFax', 'en': 'Fax Provider'},
 {'cs': 'DatumZahajeniCinnosti', 'en': 'Date Started Activities'},
 {'cs': 'IdentifikatorDatoveSchranky', 'en': 'Mailbox Identifier'},
 {'cs': 'PoskytovatelEmail', 'en': 'Email Provider'},
 {'cs': 'PoskytovatelWeb

We are interested matching the input columns of `cz` with the output template, as listed in the configuration attribute `config['index']`, namely:

In [20]:
print("\033[4mList of output/template columns:\033[0m")
for (k,v) in cz.config['index'].items():
    print("- '\033[1m%s\033[0m' (shortcut: %s)" % (v['name'],k))

List of output/template columns:
- 'id' (shortcut: id)
- 'hospital_name' (shortcut: name)
- 'site_name' (shortcut: site)
- 'lat' (shortcut: lat)
- 'lon' (shortcut: lon)
- 'geo_qual' (shortcut: geo_qual)
- 'street' (shortcut: street)
- 'house_number' (shortcut: number)
- 'postcode' (shortcut: postcode)
- 'city' (shortcut: city)
- 'cc' (shortcut: cc)
- 'country' (shortcut: country)
- 'cap_beds' (shortcut: beds)
- 'cap_prac' (shortcut: prac)
- 'cap_rooms' (shortcut: rooms)
- 'emergency' (shortcut: ER)
- 'facility_type' (shortcut: type)
- 'public_private' (shortcut: PP)
- 'list_specs' (shortcut: specs)
- 'tel' (shortcut: tel)
- 'email' (shortcut: email)
- 'url' (shortcut: url)
- 'ref_date' (shortcut: refdate)
- 'pub_date' (shortcut: pubdate)
- 'comments' (shortcut: comments)


## Fit the input to the output configuration format<a id="matching"></a>

In practice, we will match the names of the input fields (whatever the language, as stored in the `cols` attribute) with the shortcut names of the `cz.config['index']` attribute (as listed above) through an educated guess regarding their actual meaning. For instance, we identify the following correspondance (subject to changes owing to the quality of the automated translation...):

In [21]:
matched_columns = {'id':       'Medical Equipment Id', 
                   'name':     'Medical Equipment Id', 
                   'site':     'The name of the cell', 
                   'lat':      'GPS', 
                   'lon':      'GPS', 
                   'street':   'Street', 
                   'number':   'Home Number Orientation', 
                   'postcode': 'Zip code', 
                   'city':     'Village', 
                   'email':    'Email Provider', 
                   'pubdate':  'Last Modified'}

Obviously, it is always better to have a native to double check the correspondance...

All we need to do to operate the matching/extraction is to load the information regarding `matched_columns` into the index matching attribute `idx` of the `cz` instance. The attribute `cz.idx` will therefore contains the direct one-to-one matching (simple reassignment + format cast) between columns that do not require complex transformation (except for the `'lat', 'lon'` fields, see below): 

In [22]:
cz.idx.update(matched_columns)

print("\033[4mIndexing dictionary:\033[0m")
pprint(cz.idx)

Indexing dictionary:
{'city': 'Village',
 'email': 'Email Provider',
 'id': 'Medical Equipment Id',
 'lat': 'GPS',
 'lon': 'GPS',
 'name': 'Medical Equipment Id',
 'number': 'Home Number Orientation',
 'postcode': 'Zip code',
 'pubdate': 'Last Modified',
 'site': 'The name of the cell',
 'street': 'Street'}


Note in the above that the `'lat'` and `'lon'` are both "matched" to the `'GPS'` column (not translated) of the input dataset since its actually concatenates (in a string) the geographical coordinates. The `locate_data` method (see below) will help handling these specific fields:

In [23]:
for l in ['lat', 'lon']:
    print("- Coordinate \033[1m'%s'\033[0m is represented in the column \033[1m'%s'\033[0m of the input table" 
          % (l, cz.idx[l])) 

cz.data['GPS'].head(5)

- Coordinate 'lat' is represented in the column 'GPS' of the input table
- Coordinate 'lon' is represented in the column 'GPS' of the input table


0    49.427787867131 15.218289906425
1    50.497217046878 13.650265371038
2    50.073416715218 14.401090995315
3    50.126760922745 14.456956313382
4     49.183499865155 15.45981410537
Name: GPS, dtype: object

Following, we run the subsequent geolocation and formatting operations.
* `locate_data` is used to retrieve the geographical  `('lat','lon')` coordinates based on the input geolocation information available in `'GPS'`; because no argument is parsed, default/intrinsic parameters available through the properties of the `cz` dataset are considered: 

In [24]:
cz.locate_data()

* `format_data`  to perform the input matching and output formatting of the different columns of the dataset; `keep` is set to `True` so that all fields listed in the template configuration are represented:

In [25]:
cz.format_data(keep = True)

Check the output data, _e.g._ let's have a look at the column names and the content of the table:

In [26]:
assert (set(cz.data.columns)
        .difference(set([ind['name'] for ind in cz.config['index'].values()] + ['place']))
       ) == set()

print("\033[4mOutput data columns:\033[0m\n\033[1m%s\033[0m" % list(cz.data.columns))

cz.data.head(5)

Output data columns:
['lat', 'lon', 'geo_qual', 'country', 'cc', 'id', 'hospital_name', 'site_name', 'street', 'house_number', 'postcode', 'city', 'email', 'pub_date']


,lat,lon,geo_qual,country,cc,id,hospital_name,site_name,street,house_number,postcode,city,email,pub_date
0,49.427788,15.218290,1,Czechia,CZ,138130,138130,Zubní studio V+V s.r.o.,Komenského,1465,39301.0,Pelhøimov,nan,01-02-2020 23:03
1,50.497217,13.650265,1,Czechia,CZ,138129,138129,TECTUM spol. s r.o.,Topolová,1234,43401.0,Most,operator@labin.cz,01-02-2020 23:03
2,50.073417,14.401091,1,Czechia,CZ,138128,138128,"MUDr. Milan Kuèera, s.r.o.",Kartouzská,3274/10,15000.0,Praha,milankucera@seznam.cz,01-02-2020 23:03
3,50.126761,14.456956,1,Czechia,CZ,138127,138127,Tereza Horáèková,Klapkova,154/46,18200.0,Praha,nan,01-02-2020 23:03
4,49.183500,15.459814,1,Czechia,CZ,138126,138126,Veobecný lékaø Prevent s.r.o.,Masarykova,330,58856.0,Telè,nan,01-02-2020 23:03


See in particular the `lat`/`lon` attributes that were retrieved:

In [27]:
cz.data[['lat', 'lon']].head(5)

,lat,lon
0,49.427788,15.218290
1,50.497217,13.650265
2,50.073417,14.401091
3,50.126761,14.456956
4,49.183500,15.459814


## Store and expose the output harmonised data<a id="store-data"></a>

You can for instance transform the data into a *GEOJSON* collection of features and retrieve the output geometry using the `dumps_data` method:

In [28]:
try:
    assert False
    cz.save_data(fmt='geojson', latlon = ['lat', 'lon'])
except AssertionError:
    print("Nothing saved - set to True to actually generate the output files")

Nothing saved - set to True to actually generate the output files


Say you are interested in the 10 first facilities in the list, you can also use:

In [29]:
# columns = set(cz.data.columns).difference(set(['lat', 'lon']))
geom = io.Frame.to_geojson(cz.data.iloc[0:10,:], latlon = ['lat', 'lon'])
print("The first feature of the collection is:")
pprint(geom.get('features',[])[0])

The first feature of the collection is:
{'geometry': {'coordinates': [15.218289906425, 49.427787867131],
              'type': 'Point'},
 'properties': {'cc': 'CZ',
                'city': 'Pelhøimov',
                'country': 'Czechia',
                'email': 'nan',
                'geo_qual': 1,
                'hospital_name': '138130',
                'house_number': '1465',
                'id': 138130,
                'postcode': '39301.0',
                'pub_date': '01-02-2020 23:03',
                'site_name': 'Zubní studio V+V s.r.o.',
                'street': 'Komenského'},
 'type': 'Feature'}


You can use the formatted data to represent the available information about the 10 records you selected on a map, using any of the [`folium`](https://python-visualization.github.io/folium/) or [`ipyleaflet`](https://github.com/jupyter-widgets/ipyleaflet) packages:

In [30]:
try:
    from folium import Map, GeoJson, features
except:
    from ipywidgets import HTML
    from ipyleaflet import Map, GeoJSON, WidgetControl

In [31]:
prague = [50.0755, 14.4378]
zoom = 7
try:
    m = Map(location = prague, zoom_start = zoom)
    GeoJson(json.dumps(geom), name = 'CZ hospital',
        tooltip=features.GeoJsonTooltip(fields=['site_name','city','postcode'], localize=True)).add_to(m)
except:
    m = Map(center = prague, zoom = zoom)
    geo_json = GeoJSON(data=geom)
    m.add_layer(geo_json)
    html = HTML("CZ hospital")
    m.add_control(WidgetControl(widget=html, position='bottomleft'))    
    def on_hover(feature, **kwargs):
        html.value = '''<h4><b>{}</b></h4> \br {} {}'''.format(feature['properties']['site_name'], feature['properties']['city'], feature['properties']['postcode'])
    geo_json.on_hover(on_hover)
m

If instead, you want to save the data in an output file on disk, for instance in *CSV* or *GEOJSON* formats:

In [32]:
try:
    assert False
    cz.save_data('CZ.csv', fmt='csv')
except AssertionError:
    print("Nothing saved - set to True to actually generate the output files")

Nothing saved - set to True to actually generate the output files


## Set and share the formatting metadata<a id="store-metadata"></a>

Besides the output data, the input metadata/configuration templates are of interest to ensure the reproducibility and reuse of the operation, but also to support the simplifcation of the processes.

The configuration template providing with the formatting of output harmonised data for any given facility, can be shared. For instance, the template for *HCS* facilities `config.FACMETADATA['HCS']` that is used for the country-specific *CZ* case and is also stored in the `config` attribute of the `cz` instance (see above), can be reused for other countries: 

In [33]:
print("\033[4mFacility-generic configuration template:\033[0m")
pprint(cz.config.to_dict())

Facility-generic configuration template:
{'category': {'code': 'hcs', 'name': 'Healthcare services'},
 'facil': 'HCS',
 'index': {'ER': {'desc': "Flag 'yes/no' for whether the healthcare site "
                          'provides emergency medical services',
                  'name': 'emergency',
                  'type': 'str',
                  'values': ['yes', 'no']},
           'PP': {'desc': "Status 'private/public' of the healthcare service",
                  'name': 'public_private',
                  'type': 'str',
                  'values': ['public', 'private']},
           'beds': {'desc': 'Measure of capacity by number of beds (most '
                            'common)',
                    'name': 'cap_beds',
                    'type': 'int',
                    'values': None},
           'cc': {'desc': 'Country code (ISO 3166-1 alpha-2 format)',
                  'name': 'cc',
                  'type': 'str',
                  'values': ['BE',
                     

Similarly to the facility-generic metadata above, it is possible to retrieve the country-specific metadata, *i.e.* all relevant information regarding the `cz` dataset. Actually, the `cz` instance currently contains no metadata, but the `meta` attribute should be updated:

In [34]:
try:
    assert cz.meta != {}
except:
    print('Further information regarding data loading is requested...')
    cz.update_meta()

assert cz.meta == cz.get_meta(keys = True, force = True)

print("\033[4mCountry-specific metadata:\033[0m")
pprint(cz.meta.to_dict())

Further information regarding data loading is requested...
Country-specific metadata:
{'category': {'code': 'hcs', 'name': 'Healthcare services'},
 'columns': [{'cs': 'ZdravotnickeZarizeniId', 'en': 'Medical Equipment Id'},
             {'cs': 'PCZ', 'en': 'PCZ'},
             {'cs': 'PCDP', 'en': 'PCDP'},
             {'cs': 'NazevCely', 'en': 'The name of the cell'},
             {'cs': 'DruhZarizeni', 'en': 'Type of device'},
             {'cs': 'Obec', 'en': 'Village'},
             {'cs': 'Psc', 'en': 'Zip code'},
             {'cs': 'Ulice', 'en': 'Street'},
             {'cs': 'CisloDomovniOrientacni', 'en': 'Home Number Orientation'},
             {'cs': 'Kraj', 'en': 'Region'},
             {'cs': 'KrajCode', 'en': 'County Code'},
             {'cs': 'Okres', 'en': 'District'},
             {'cs': 'OkresCode', 'en': 'District Code'},
             {'cs': 'SpravniObvod', 'en': 'Administrative District'},
             {'cs': 'PoskytovatelTelefon', 'en': 'Phone Provider'},
       

and save these metadata in a specific dedicated file on disk:

In [35]:
CZMETA = 'temp_czmeta.json'
try:
    cz.save_meta(CZMETA, fmt='json', force = True)
except AssertionError:
    print("Nothing saved - set to True to actually generate the output files")

## Adopt a metadata-based approach<a id="metadata-approach"></a>

Since the metadata contains all the information necessary to process the source dataset, it shall be used _as is_ to operate the harmonisation of the data.

Metadata can easily be retrieved then: 

In [36]:
with open(CZMETA, 'r') as fp:
    metadata = json.load(fp)
metadata = config.MetaDatNatFacility(metadata)

assert metadata.to_dict() == cz.meta.to_dict()

Following, like in the section above, a dynamically class `NewCZhcs` can be created and configured with this information:

In [37]:
NewCZhcs = config.facilityFactory(cz.config.to_dict(), # facility = 'HCS', 
                                  meta = metadata )

then, similarly, a new instance can be created...and that's all we will need to process the data:

In [38]:
newcz = NewCZhcs()

Given the `file` and `path` fields of the `newcz.meta` attribute variable, data can be loaded using the `load_data` method (without argument, since they are all intrinsically provided with the metadata). But also given the `index` field, the formatting using the `format_data` method can be operated directly:

In [39]:
newcz.load_data()
newcz.locate_data()
newcz.format_data()

Check the output data:

In [40]:
print("\033[4mOutput data columns:\033[0m\n\033[1m%s\033[0m" % list(newcz.data.columns))

newcz.data.head(5)

Output data columns:
['lat', 'lon', 'geo_qual', 'country', 'cc', 'id', 'hospital_name', 'site_name', 'street', 'house_number', 'postcode', 'city', 'email', 'pub_date']


,lat,lon,geo_qual,country,cc,id,hospital_name,site_name,street,house_number,postcode,city,email,pub_date
0,49.427788,15.218290,1,Czechia,CZ,138130,138130,Zubní studio V+V s.r.o.,Komenského,1465,39301.0,Pelhøimov,nan,01-02-2020 23:03
1,50.497217,13.650265,1,Czechia,CZ,138129,138129,TECTUM spol. s r.o.,Topolová,1234,43401.0,Most,operator@labin.cz,01-02-2020 23:03
2,50.073417,14.401091,1,Czechia,CZ,138128,138128,"MUDr. Milan Kuèera, s.r.o.",Kartouzská,3274/10,15000.0,Praha,milankucera@seznam.cz,01-02-2020 23:03
3,50.126761,14.456956,1,Czechia,CZ,138127,138127,Tereza Horáèková,Klapkova,154/46,18200.0,Praha,nan,01-02-2020 23:03
4,49.183500,15.459814,1,Czechia,CZ,138126,138126,Veobecný lékaø Prevent s.r.o.,Masarykova,330,58856.0,Telè,nan,01-02-2020 23:03


## Automated metadata-based harmonisation<a id="harmonisation"></a>

Obviously, you will not have to rerun these operations everytime a table is created. 

In order to simplify even further the operations, the steps above, including the loading of configuration and metadata files, have been automated. You can run it at once using the `harmonise` module:

In [41]:
from pyeufacility import harmonise
cz = harmonise.run('HCS', country = "CZ", on_disk = True, dest = 'CZtest.csv')

! Country py-module 'pyeufacility.hcs.CZhcs' found !
! Generic formatting/harmonisation methods used !
! Harmonised data for country 'CZ' generated !


Check again for consistency... :

In [42]:
cz.data[['lat', 'lon']].head(5)

,lat,lon
0,49.427788,15.218290
1,50.497217,13.650265
2,50.073417,14.401091
3,50.126761,14.456956
4,49.183500,15.459814


## Automated metadata-based validation<a id="validation"></a>

Last, you will want to validate the output data, given the information you provided in the configuration template (_e.g._, through the field `column`).

Validation is also an automated process:

In [43]:
from pyeufacility import validate
validate.run('HCS', country = "CZ", src = 'CZtest.csv')

! Missing columns in source file: '['facility_type', 'cap_prac', 'emergency', 'comments', 'public_private', 'cap_beds', 'ref_date', 'list_specs', 'tel', 'url', 'cap_rooms']' !


! Data passed validation (see warning reports) !
